In [ ]:
# Imports

import numpy as np
from tensorflow.keras.datasets import fashion_mnist, mnist
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sn
import wandb

In [ ]:
# Creating train and test data

(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
X_train_new = X_train.reshape(
    (X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
X_test_new = X_test.reshape(
    (X_test.shape[0], X_test.shape[1] * X_test.shape[2]))

In [ ]:
# The RBN class

def sig(z):
    return np.where(z > 0, 1 / (1 + np.exp(-z)), np.exp(z) / (1 + np.exp(z)))

class RBN:
    
    def __init__(self, hidden_layer_dim = 64, k = 200, r = 10, learning_rate = 0.01, epochs = 1, sampling = 'Gibbs'):
        self.input_dim = None
        self.hidden_layer_dim = hidden_layer_dim
        self.k = k
        self.r = 10
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.sampling = sampling
    
    def prob_v(self, h):
        return sig((h @ self.w.T) + self.b)
    
    def prob_h(self, x):
        return sig((x @ self.w) + self.c)
    
    def get_hidden(self, X):
        probs = sig(np.sign((X @ self.w) + self.c))
        return np.random.binomial(1, probs, probs.shape)
    
    def single_update(self, x):
        v = x
        hid = np.zeros((self.hidden_layer_dim,))
        
        for t in range(0, self.k):
            hid = np.random.binomial(1, self.prob_h(v), self.hidden_layer_dim)
            v = np.random.binomial(1, self.prob_v(hid), self.input_dim)
        
        v_list = []
        
        for t in range(0, self.r):
            hid = np.random.binomial(1, self.prob_h(v), self.hidden_layer_dim)
            v = np.random.binomial(1, self.prob_v(hid), self.input_dim)
            v_list.append(v)
            
        all_v = np.array(v_list)
        transf_all_v = sig((all_v @ self.w) + self.c)
        mat_all_v = all_v.reshape((self.r, self.input_dim, 1)) @ transf_all_v.reshape((self.r, 1, self.hidden_layer_dim))
        
        transf_x = sig((x @ self.w) + self.c)
        mat_x = x.reshape((self.input_dim, 1)) @ transf_x.reshape((1, self.hidden_layer_dim))
        
        self.w += self.learning_rate * (mat_x - mat_all_v.mean(axis=0))
        self.b += self.learning_rate * (x - all_v.mean(axis=0))
        self.c += self.learning_rate * (transf_x - transf_all_v.mean(axis=0))
        
    def weights_init(self):
        self.b = np.zeros((self.input_dim,))
        self.c = np.zeros((self.hidden_layer_dim,))
        self.w = np.zeros((self.input_dim, self.hidden_layer_dim))
                    
    def learn(self, X):
        self.input_dim = X.shape[1]
        self.weights_init()
        
        for ep in range(self.epochs):
            for x in X:
                self.single_update(x)
        
        